## Google's Word2Vec for movie reviews

https://www.kaggle.com/c/word2vec-nlp-tutorial  
https://github.com/wendykan/DeepLearningMovies  
http://fastml.com/classifying-text-with-bag-of-words-a-tutorial/  


In this tutorial competition, we dig a little "deeper" into sentiment analysis. Google's Word2Vec is a deep-learning inspired method that focuses on the meaning of words. Word2Vec attempts to understand meaning and semantic relationships among words. It works in a way that is similar to deep approaches, such as recurrent neural nets or deep neural nets, but is computationally more efficient. This tutorial focuses on Word2Vec for sentiment analysis.

Sentiment analysis is a challenging subject in machine learning. People express their emotions in language that is often obscured by sarcasm, ambiguity, and plays on words, all of which could be very misleading for both humans and computers. There's another Kaggle competition for movie review sentiment analysis. In this tutorial we explore how Word2Vec can be applied to a similar problem.

Deep learning has been in the news a lot over the past few years, even making it to the front page of the New York Times. These machine learning techniques, inspired by the architecture of the human brain and made possible by recent advances in computing power, have been making waves via breakthrough results in image recognition, speech processing, and natural language tasks. Recently, deep learning approaches won several Kaggle competitions, including a drug discovery task, and cat and dog image recognition.

#### Let's charge the batteries for our analysis...

In [105]:
import os
import sys
import time
import re
import pickle
import logging

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pylab
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
#import seaborn as sns
#import seaborn.apionly as sns

import nltk
import nltk.data
from nltk.corpus import stopwords

from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import roc_auc_score as AUC

from gensim.models import Word2Vec

%matplotlib inline
#%pylab inline

Data Set  
--

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.  

testData - The test set. The tab-delimited file has a header row followed by 25,000 rows containing an id and text for each review. Your task is to predict the sentiment for each one. 

unlabeledTrainData - An extra training set with no labels. The tab-delimited file has a header row followed by 50,000 rows containing an id and text for each review. 

sampleSubmission - A comma-delimited sample submission file in the correct format.
Data fields

id - Unique ID of each review
sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
review - Text of the review

Loading the dataset:  
--

In [2]:
datapath = "/home/rsouza/Dropbox/Renato/ModMinDados/datasets/Kaggle/BoW_Pop_Corn/"
outputs = "/home/rsouza/Dropbox/Renato/ModMinDados/outputs/"

In [3]:
train = pd.read_csv(os.path.join(datapath, 'labeledTrainData.tsv'), header=0, delimiter="\t", quoting=3)
test = pd.read_csv(os.path.join(datapath, 'testData.tsv'), header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv(os.path.join(datapath, "unlabeledTrainData.tsv"), header=0, delimiter="\t", quoting=3)

In [4]:
print("Read %d labeled train reviews, %d labeled test reviews, and %d unlabeled reviews\n") % (train["review"].size,
                                                                                               test["review"].size,
                                                                                               unlabeled_train["review"].size)

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [5]:
train.info()
#train.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25000 entries, 0 to 24999
Data columns (total 3 columns):
id           25000 non-null object
sentiment    25000 non-null int64
review       25000 non-null object
dtypes: int64(1), object(2)
memory usage: 781.2+ KB


In [6]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


Part 1: For Beginners - Bag of Words
--
https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words  

What is NLP?

NLP (Natural Language Processing) is a set of techniques for approaching text problems. This page will help you get started with loading and cleaning the IMDB movie reviews, then applying a simple Bag of Words model to get surprisingly accurate predictions of whether a review is thumbs-up or thumbs-down.

In [21]:
class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist(review, remove_stopwords=False):
        # Function to convert a document to a sequence of words, optionally removing stop words.  
        # Returns a list of words.
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        # 5. Return a list of words
        return(words)

    @staticmethod
    def review_to_sentences(review, tokenizer, remove_stopwords=False):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append(KaggleWord2VecUtility.review_to_wordlist(raw_sentence, remove_stopwords))
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences

Cleaning all the datasets and getting word lists
--
first set is without stopwords

In [8]:
clean_train_reviews = []
for i in xrange(0, len(train["review"])):
    clean_train_reviews.append(" ".join(KaggleWord2VecUtility.review_to_wordlist(train["review"][i], True)))
clean_train_reviews[100]

u'uk edition show rather less extravagant us version person concerned get new kitchen perhaps bedroom bathroom wonderfully grateful got us version show everything reality tv shouldn instead making improvements house occupants could afford entire house gets rebuilt know show trying show lousy welfare system exists us beg hard enough receive rather vulgar product placement takes place particularly sears also uncalled rsther turning one family deprived area potential millionaires would far better help community whole instead spending hundreds thousands dollars one home build something whole community perhaps place diy power tools borrowed returned along building materials everyone benefit want giving one person cause enormous resentment among rest local community still live run houses'

In [9]:
clean_test_reviews = []
for i in xrange(0,len(test["review"])):
    clean_test_reviews.append(" ".join(KaggleWord2VecUtility.review_to_wordlist(test["review"][i], True)))
clean_test_reviews[100]

u'one true thing quiet film opened fall glowing reviews mild box office tells crippled story ellen renee zellweger workaholic forced move back home take care terminally ill mother meryl streep aloof father william hurt run academic department terms general strength one true thing lies way actors elevate characters hollywood clich territory streep kate perfect homemaker whose ability light room charm evident opening scenes costume party celebrating hurt birthday ellen never close mother since graduated harvard university certain destain ellen almost thinks mother simplistic air head hand admires father shares special passion writing ellen writes aggressive new york firm almost heartbroken latest piece torn hurt seems lonely figure get point kate gets sicker ellen perspectives change grows closer mother distant father hurt keeps making excuses family needs ellen assumes affair meanwhile given desk work spend time craft activities mother cult group minnies also learning mother isn weak fi

Saving pickles for posterior usage  

In [10]:
with open(os.path.join(outputs, 'clean_reviews.pkl'),'w') as f:
    pickle.dump((clean_train_reviews, 
                 clean_test_reviews),f)

Second set mantains stopwords

In [11]:
clean_train_reviews_sw = []
for i in xrange(0, len(train["review"])):
    clean_train_reviews_sw.append(" ".join(KaggleWord2VecUtility.review_to_wordlist(train["review"][i], False)))
    
clean_train_reviews_sw[100]

u'there is a uk edition to this show which is rather less extravagant than the us version the person concerned will get a new kitchen or perhaps bedroom and bathroom and is wonderfully grateful for what they have got the us version of this show is everything that reality tv shouldn t be instead of making a few improvements to a house which the occupants could not afford or do themselves the entire house gets rebuilt i do not know if this show is trying to show what a lousy welfare system exists in the us or if you beg hard enough you will receive the rather vulgar product placement that takes place particularly by sears is also uncalled for rsther than turning one family in a deprived area into potential millionaires it would be far better to help the community as a whole where instead of spending the hundreds of thousands of dollars on one home build something for the whole community perhaps a place where diy and power tools can be borrowed and returned along with building materials s

In [12]:
clean_test_reviews_sw = []
for i in xrange(0,len(test["review"])):
    clean_test_reviews_sw.append(" ".join(KaggleWord2VecUtility.review_to_wordlist(test["review"][i], False)))
    
clean_test_reviews_sw[100]

u'one true thing is a very quiet film that opened in the fall of to glowing reviews but mild box office it tells the crippled story of ellen renee zellweger a workaholic who is forced to move back home to take care of her terminally ill mother meryl streep so that her aloof father william hurt can run his academic department these terms are only general the strength of one true thing lies in the way the actors elevate their characters above hollywood clich territory streep is kate the perfect homemaker whose ability to light up a room with her charm is evident in her opening scenes at a costume party celebrating hurt s birthday but ellen has never been close to her mother and since she graduated from harvard university has a certain destain about her ellen almost thinks her mother is a simplistic air head while on the other hand she admires her father who shares a special passion writing ellen writes for an aggressive new york firm and is almost heartbroken when her latest piece is tor

Saving pickles for posterior usage 

In [13]:
with open(os.path.join(outputs, 'clean_reviews_sw.pkl'),'w') as f:
    pickle.dump((clean_train_reviews_sw, 
                 clean_test_reviews_sw),f)

Loading pickles  

In [ ]:
with open(os.path.join(outputs, 'clean_reviews.pkl'),'r') as f:
    clean_train_reviews, clean_test_reviews = pickle.load(f)

with open(os.path.join(outputs, 'clean_reviews_sw.pkl'),'r') as f:
    clean_train_reviews_sw, clean_test_reviews_sw = pickle.load(f)

Creating Features from a Bag of Words (Using scikit-learn)
--

Now that we have our training reviews tidied up, how do we convert them to some kind of numeric representation for machine learning? One common approach is called a Bag of Words. The Bag of Words model learns a vocabulary from all of the documents, then models each document by counting the number of times each word appears. For example, consider the following two sentences:

Sentence 1: "The cat sat on the hat"  
Sentence 2: "The dog ate the cat and the hat"  

From these two sentences, our vocabulary is as follows:

{ the, cat, sat, on, hat, dog, ate, and }

To get our bags of words, we count the number of times each word occurs in each sentence. In Sentence 1, "the" appears twice, and "cat", "sat", "on", and "hat" each appear once, so the feature vector for Sentence 1 is:

{ the, cat, sat, on, hat, dog, ate, and }

Sentence 1: { 2, 1, 1, 1, 1, 0, 0, 0 }

Similarly, the features for Sentence 2 are: { 3, 1, 0, 0, 1, 1, 1, 1}

In the IMDB data, we have a very large number of reviews, which will give us a large vocabulary. To limit the size of the feature vectors, we should choose some maximum vocabulary size. Below, we use the 5000 most frequent words (remembering that stop words have already been removed).

We'll be using the feature_extraction module from scikit-learn to create bag-of-words features.  

We will test two strategies: CountVectorizer and Tfidf Vectorizer:

In [14]:
# Initialize the "CountVectorizer" object, which is scikit-learn's bag of words tool.
#http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
vectorizer1 = CountVectorizer(analyzer = "word", 
                              tokenizer = None,
                              preprocessor = None,
                              stop_words = None,
                              max_features = 5000)

In [15]:
#Another approach using TfIDf vectorizer and using the texts with stopwords in:
#https://github.com/zygmuntz/classifying-text/blob/master/bow_predict.py 
vectorizer2 = TfidfVectorizer(max_features = 6000,
                              ngram_range = (1, 3), 
                              sublinear_tf = True,)

Fit_transform() does two functions: First, it fits the model and learns the vocabulary; 
second, it transforms our training data into feature vectors. 
The input to fit_transform should be a list of strings.

In [16]:
train_data_features1 = vectorizer1.fit_transform(clean_train_reviews)
train_data_features1 = train_data_features1.toarray() # Numpy arrays are easy to work with
print(train_data_features1.shape)

(25000, 5000)


In [17]:
with open(os.path.join(outputs, 'train_data_features1.pkl'),'w') as f:
    pickle.dump(train_data_features1,f)

In [8]:
with open(os.path.join(outputs, 'train_data_features1.pkl'),'r') as f:
    train_data_features1 = pickle.load(f)

In [18]:
train_data_features2 = vectorizer2.fit_transform(clean_train_reviews_sw)
train_data_features2 = train_data_features2.toarray() # Numpy arrays are easy to work with
print(train_data_features2.shape)

(25000, 6000)


In [ ]:
with open(os.path.join(outputs, 'train_data_features2.pkl'),'w') as f:
    pickle.dump(train_data_features2,f)

In [32]:
with open(os.path.join(outputs, 'train_data_features2.pkl'),'r') as f:
    train_data_features2 = pickle.load(f)

In [19]:
test_data_features1 = vectorizer1.fit_transform(clean_test_reviews)
test_data_features1 = test_data_features1.toarray() # Numpy arrays are easy to work with
print(test_data_features1.shape)

(25000, 5000)


In [ ]:
with open(os.path.join(outputs, 'test_data_features1.pkl'),'w') as f:
    pickle.dump(test_data_features1,f)

In [ ]:
with open(os.path.join(outputs, 'test_data_features1.pkl'),'r') as f:
    test_data_features1 = pickle.load(f)

In [20]:
test_data_features2 = vectorizer2.fit_transform(clean_test_reviews_sw)
test_data_features2 = test_data_features2.toarray() # Numpy arrays are easy to work with
print(test_data_features2.shape)

(25000, 6000)


In [ ]:
with open(os.path.join(outputs, 'test_data_features2.pkl'),'w') as f:
    pickle.dump(test_data_features2,f)

In [50]:
with open(os.path.join(outputs, 'test_data_features2.pkl'),'r') as f:
    test_data_features2 = pickle.load(f)

Dividing Train set for Cross Validation  
--

https://github.com/zygmuntz/classifying-text/blob/master/bow_validate.py  
Alternatively, we can use the indexes to divide the train samples  

train_i, test_i = train_test_split(np.arange(len(train)), train_size = 0.8, random_state = 44)  

After generating indexes, we can divide ou datasets:  
traincv = train_data_features1[train_i]  
testcv = train_data_features1[test_i]

In [34]:
X_traincv1, X_testcv1, y_traincv1, y_testcv1 = train_test_split(train_data_features1,
                                                            train["sentiment"],
                                                            test_size=0.2,
                                                            random_state=0)

In [33]:
X_traincv2, X_testcv2, y_traincv2, y_testcv2 = train_test_split(train_data_features2,
                                                            train["sentiment"],
                                                            test_size=0.2,
                                                            random_state=0)

Let's create a helper function to evaluate the classifiers:  

In [18]:
def mean_scores_cv(clf, cv, X, y):
    scores = cross_validation.cross_val_score(clf, X, y, 
                                              scoring=None, 
                                              cv=cv, 
                                              n_jobs=-1,
                                              verbose=0,
                                              fit_params=None,
                                              pre_dispatch='2*n_jobs')
    return scores.mean()

Training some Classifiers  
--

Random Forest
--

At this point, we have numeric training features from the Bag of Words and the original sentiment labels for each feature vector, so let's do some supervised learning! Here, we'll use the Random Forest classifier.  The Random Forest algorithm is included in scikit-learn (Random Forest uses many tree-based classifiers to make predictions, hence the "forest"). Below, we set the number of trees to 100 as a reasonable default value. More trees may (or may not) perform better, but will certainly take longer to run. Likewise, the more features you include for each review, the longer this will take.

In [36]:
# Initialize a Random Forest classifier with 100 trees
clf_RF_1 = RandomForestClassifier(n_estimators=100, 
                                  criterion='gini', 
                                  max_depth=None, 
                                  min_samples_split=2, 
                                  min_samples_leaf=1, 
                                  min_weight_fraction_leaf=0.0, 
                                  max_features='auto', 
                                  max_leaf_nodes=None, 
                                  bootstrap=True, 
                                  oob_score=False, 
                                  n_jobs=1, 
                                  random_state=None, 
                                  verbose=0, 
                                  warm_start=False, 
                                  class_weight=None).fit(X_traincv1, y_traincv1)

In [37]:
eval_RF_1_tts = clf_RF_1.score(X_testcv1, y_testcv1)
print(eval_RF_1_tts)

0.8508


In [45]:
clf_RF_1.predict_proba(testcv1)

array([[ 0.53,  0.47],
       [ 0.73,  0.27],
       [ 0.07,  0.93],
       ..., 
       [ 0.13,  0.87],
       [ 0.29,  0.71],
       [ 0.19,  0.81]])

In [46]:
# Initialize a Random Forest classifier with 100 trees
clf_RF_2 = RandomForestClassifier(n_estimators=100, 
                                  criterion='gini', 
                                  max_depth=None, 
                                  min_samples_split=2, 
                                  min_samples_leaf=1, 
                                  min_weight_fraction_leaf=0.0, 
                                  max_features='auto', 
                                  max_leaf_nodes=None, 
                                  bootstrap=True, 
                                  oob_score=False, 
                                  n_jobs=1, 
                                  random_state=None, 
                                  verbose=0, 
                                  warm_start=False, 
                                  class_weight=None).fit(X_traincv2, y_traincv2)

In [47]:
eval_RF_2_tts = clf_RF_2.score(X_testcv2, y_testcv2)
print(eval_RF_2_tts)

0.853


Logistic Regression  
--

In [41]:
#http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

clf_LR_1 = LR(penalty='l2',
              dual=False,
              tol=0.0001,
              C=1.0,
              fit_intercept=True,
              intercept_scaling=1,
              class_weight=None,
              random_state=None,
              solver='liblinear',
              max_iter=100,
              multi_class='ovr',
              verbose=0).fit(X_traincv1, y_traincv1)

In [42]:
eval_LR_1_tts = clf_LR_1.score(X_testcv1, y_testcv1)
print(eval_LR_1_tts)

0.8586


In [43]:
clf_LR_2 = LR(penalty='l2',
              dual=False,
              tol=0.0001,
              C=1.0,
              fit_intercept=True,
              intercept_scaling=1,
              class_weight=None,
              random_state=None,
              solver='liblinear',
              max_iter=100,
              multi_class='ovr',
              verbose=0).fit(X_traincv2, y_traincv2)

In [44]:
eval_LR_2_tts = clf_LR_2.score(X_testcv2, y_testcv2)
print(eval_LR_2_tts)

0.8926


Creating a Submission  
--

All that remains is to run the trained Random Forest on our test set and create a submission file. If you haven't already done so, download testData.tsv from the Data page. This file contains another 25,000 reviews and ids; our task is to predict the sentiment label.

Note that when we use the Bag of Words for the test set, we only call "transform", not "fit_transform" as we did for the training set. In machine learning, you shouldn't use the test set to fit your model, otherwise you run the risk of overfitting. For this reason, we keep the test set off-limits until we are ready to make predictions.

In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25000 entries, 0 to 24999
Data columns (total 2 columns):
id        25000 non-null object
review    25000 non-null object
dtypes: object(2)
memory usage: 585.9+ KB


In [13]:
test.head()

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [72]:
# Use the random forest to make sentiment label predictions
result = clf_LR_2.predict(test_data_features2)
result_prob = clf_LR_2.predict_proba(test_data_features2)
output = pd.DataFrame(data={"id":test["id"], "sentiment":result,})# "probs":result_prob[:,1]})
# Use pandas to write the comma-separated output file
output.to_csv(os.path.join(datapath,'Bag_of_Words_model.csv'), index=False, quoting=3)

In [74]:
output.head()

,id,probs,sentiment
0,"""12311_10""",0.659651,1
1,"""8348_2""",0.895754,1
2,"""5828_4""",0.222182,0
3,"""7186_2""",0.479960,0
4,"""12128_7""",0.480401,0


Part 2: Word Vectors
--

https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-2-word-vectors  

Introducing Distributed Word Vectors: This part of the tutorial will focus on using distributed word vectors created by the Word2Vec algorithm.

Word2vec, published by Google in 2013, is a neural network implementation that learns distributed representations for words. Other deep or recurrent neural network architectures had been proposed for learning word representations prior to this, but the major problem with these was the long time required to train the models. Word2vec learns quickly relative to other models.

Word2Vec does not need labels in order to create meaningful representations. This is useful, since most data in the real world is unlabeled. If the network is given enough training data (tens of billions of words), it produces word vectors with intriguing characteristics. Words with similar meanings appear in clusters, and clusters are spaced such that some word relationships, such as analogies, can be reproduced using vector math. The famous example is that, with highly trained word vectors, "king - man + woman = queen."

Distributed word vectors are powerful and can be used for many applications, particularly word prediction and translation. Here, we will try to apply them to sentiment analysis.

Using word2vec in Python: In Python, we will use the excellent implementation of word2vec from the gensim package. If you don't already have gensim installed, you'll need to install it. There is an excellent tutorial that accompanies the Python Word2Vec implementation, here.

Although Word2Vec does not require graphics processing units (GPUs) like many deep learning algorithms, it is compute intensive. Both Google's version and the Python version rely on multi-threading (running multiple processes in parallel on your computer to save time). ln order to train your model in a reasonable amount of time, you will need to install cython (instructions here). Word2Vec will run without cython installed, but it will take days to run instead of minutes.

Preparing to Train a Model

First, we read in the data with pandas, as we did in Part 1. Unlike Part 1, we now use unlabeledTrain.tsv, which contains 50,000 additional reviews with no labels. When we built the Bag of Words model in Part 1, extra unlabeled training reviews were not useful. However, since Word2Vec can learn from unlabeled data, these extra 50,000 reviews can now be used.

The functions we write to clean the data are also similar to Part 1, although now there are a couple of differences. First, to train Word2Vec it is better not to remove stop words because the algorithm relies on the broader context of the sentence in order to produce high-quality word vectors. For this reason, we will make stop word removal optional in the functions below. It also might be better not to remove numbers, but we leave that as an exercise for the reader.

In [8]:
def getCleanReviews(reviews):
    clean_reviews = []
    for review in reviews["review"]:
        clean_reviews.append(KaggleWord2VecUtility.review_to_wordlist(review, remove_stopwords=False))
    return clean_reviews

In [9]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Split the labeled and unlabeled training sets into clean sentences
sentences = []  # Initialize an empty list of sentences
print("Parsing sentences from training set")
for review in train["review"]:
    sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
    
print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

/usr/local/lib/python2.7/dist-packages/bs4/__init__.py:169: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
/usr/local/lib/python2.7/dist-packages/bs4/__init__.py:176: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)


Parsing sentences from training set
Parsing sentences from unlabeled set

/usr/local/lib/python2.7/dist-packages/bs4/__init__.py:176: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/usr/local/lib/python2.7/dist-packages/bs4/__init__.py:176: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/usr/local/lib/

In [10]:
# Import the built-in logging module and configure it so that Word2Vec creates nice output messages
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

https://radimrehurek.com/gensim/models/word2vec.html  

Training and Saving Your Model

With the list of nicely parsed sentences, we're ready to train the model. There are a number of parameter choices that affect the run time and the quality of the final model that is produced. For details on the algorithms below, see the word2vec API documentation as well as the Google documentation. 

Architecture: Architecture options are skip-gram (default) or continuous bag of words. We found that skip-gram was very slightly slower but produced better results.

Training algorithm: Hierarchical softmax (default) or negative sampling. For us, the default worked well.

Downsampling of frequent words: The Google documentation recommends values between .00001 and .001. For us, values closer 0.001 seemed to improve the accuracy of the final model.

Word vector dimensionality: More features result in longer runtimes, and often, but not always, result in better models. Reasonable values can be in the tens to hundreds; we used 300.

Context / window size: How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax (more is better, up to a point).

Worker threads: Number of parallel processes to run. This is computer-specific, but between 4 and 6 should work on most systems.

Minimum word count: This helps limit the size of the vocabulary to meaningful words. Any word that does not occur at least this many times across all documents is ignored. Reasonable values could be between 10 and 100. In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. This resulted in an overall vocabulary size of around 15,000 words. Higher values also help limit run time.

Choosing parameters is not easy, but once we have chosen our parameters, creating a Word2Vec model is straightforward:

In [66]:
num_features = 300    # Word vector dimensionality
min_word_count = 20   # Minimum word count
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size
downsampling = 1e-3   # Downsample setting for frequent words

In [77]:
# Initialize and train the model (this will take some time)
model = Word2Vec(sentences,
                 workers = num_workers,
                 size = num_features,
                 min_count = min_word_count, 
                 window = context,
                 sample = downsampling,
                 seed=1,)

In [ ]:
#Optionally converting the model for Bigrams (to capture more context):
bigram_transformer = gensim.models.Phrases(sentences)
model = Word2Vec(bigram_transformed[sentences],
                 workers = num_workers,
                 size = num_features,
                 min_count = min_word_count, 
                 window = context,
                 sample = downsampling,
                 seed=1,)

In [78]:
# If you don't plan to train the model any further, calling init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

In [79]:
# It can be helpful to create a meaningful model name and save the model for later use. 
#You can load it later using Word2Vec.load()
model_name = "300features_20minwords_10context"
model.save(os.path.join(datapath,model_name))
model = Word2Vec.load(os.path.join(datapath,model_name))

Exploring the Model Results

Congratulations on making it successfully through everything so far! Let's take a look at the model we created out of our 75,000 training reviews.

The "doesnt_match" function will try to deduce which word in a set is most dissimilar from the others:

In [80]:
print(model.doesnt_match("man woman child kitchen".split()))
print(model.doesnt_match("france england germany berlin".split()))
print(model.doesnt_match("paris berlin london austria".split()))
print(model.most_similar("man"))
print(model.most_similar("queen"))
print(model.most_similar("awful"))

kitchen
berlin
paris
[(u'woman', 0.70549476146698), (u'boy', 0.6945683360099792), (u'guy', 0.6274200677871704), (u'lady', 0.6035640835762024), (u'girl', 0.5901005268096924), (u'young', 0.5855194926261902), (u'soldier', 0.5849857926368713), (u'himself', 0.5801713466644287), (u'tall', 0.5734527111053467), (u'person', 0.5677907466888428)]
[(u'princess', 0.695735514163971), (u'belle', 0.6137073636054993), (u'prince', 0.6068472862243652), (u'maid', 0.565276026725769), (u'rose', 0.5621911287307739), (u'marie', 0.5499277114868164), (u'bride', 0.5302281975746155), (u'elizabeth', 0.5260463953018188), (u'maria', 0.521757960319519), (u'kaurwaki', 0.5198318362236023)]
[(u'terrible', 0.8406445384025574), (u'horrible', 0.8306282758712769), (u'dreadful', 0.7595139145851135), (u'lousy', 0.7144946455955505), (u'appalling', 0.7111591696739197), (u'abysmal', 0.6997086405754089), (u'atrocious', 0.6982709765434265), (u'pathetic', 0.6962579488754272), (u'horrendous', 0.6717314124107361), (u'embarrassing', 0

So it seems we have a reasonably good model for semantic meaning - at least as good as Bag of Words. But how can we use these fancy distributed word vectors for supervised learning? The next section takes a stab at that.

Part 3: More Fun With Word Vectors
--

https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-3-more-fun-with-word-vectors  

Numeric Representations of Words

Now that we have a trained model with some semantic understanding of words, how should we use it? If you look beneath the hood, the Word2Vec model trained in Part 2 consists of a feature vector for each word in the vocabulary, stored in a numpy array called "syn0":

In [81]:
model = Word2Vec.load(os.path.join(datapath,model_name))

In [82]:
model.syn0.shape

(34119, 100)

The number of rows in syn0 is the number of words in the model's vocabulary, and the number of columns corresponds to the size of the feature vector, which we set in Part 2.  Setting the minimum word count to 40 gave us a total vocabulary of 16,492 words with 300 features apiece. Individual word vectors can be accessed in the following way:

In [83]:
model["flower"][0:10]

array([ 0.07201184, -0.06537043, -0.03638896,  0.03752942, -0.0330898 ,
        0.08165944,  0.01443508, -0.24465366, -0.07218753,  0.06249748], dtype=float32)

In [84]:
model.index2word[0:10]

[u'fawn',
 u'sonja',
 u'woods',
 u'spiders',
 u'seifer',
 u'hanging',
 u'woody',
 u'hastily',
 u'comically',
 u'disobeying']

From Words To Paragraphs, 
--
Attempt 1:  Vector Averaging  
--

One challenge with the IMDB dataset is the variable-length reviews. We need to find a way to take individual word vectors and transform them into a feature set that is the same length for every review.

Since each word is a vector in 300-dimensional space, we can use vector operations to combine the words in each review. One method we tried was to simply average the word vectors in a given review (for this purpose, we removed stop words, which would just add noise).

The following code averages the feature vectors, building on our code from Part 2. 

In [85]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given paragraph
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    # Index2word is a list that contains the names of the words in the model's vocabulary. 
    #Convert it to a set, for speed
    index2word_set = set(model.index2word)
    # Loop over each word in the review and, if it is in the model's
    # vocabulary, add its feature vector to the total
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [86]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate
    # the average feature vector for each one and return a 2D numpy array
    # Initialize a counter
    counter = 0.
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # Loop through the reviews
    for review in reviews:
        # Print a status message every 2000th review
        if counter%2000. == 0.:
            print("Review %d of %d") % (counter, len(reviews))
        #Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        counter = counter + 1.
    return reviewFeatureVecs

Let's use the pickles from Part 1  

In [24]:
with open(os.path.join(outputs, 'clean_reviews.pkl'),'r') as f:
    clean_train_reviews, 
    clean_test_reviews = pickle.load(f)

with open(os.path.join(outputs, 'clean_reviews_sw.pkl'),'r') as f:
    clean_train_reviews_sw, 
    clean_test_reviews_sw = pickle.load(f)

Now, we can call these functions to create average vectors for each paragraph. The following operations will take a few minutes:  

In [87]:
# Calculate average feature vectors for training and testing sets, using the functions 
# we defined above. Notice that we now use stop word removal.
trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)

Review 0 of 25000
Review 2000 of 25000
Review 4000 of 25000
Review 6000 of 25000
Review 8000 of 25000
Review 10000 of 25000
Review 12000 of 25000
Review 14000 of 25000
Review 16000 of 25000
Review 18000 of 25000
Review 20000 of 25000
Review 22000 of 25000
Review 24000 of 25000


Next, use the average paragraph vectors with the classifiers from Part 1.  
Note that, as in Part 1, we can only use the labeled training reviews to train the model. 

In [90]:
X_traincvWV, X_testcvWV, y_traincvWV, y_testcvWV = train_test_split(trainDataVecs,
                                                                    train["sentiment"],
                                                                    test_size=0.2,
                                                                    random_state=0)

In [91]:
# Initialize a Random Forest classifier with 100 trees
clf_RF_WV = RandomForestClassifier(n_estimators=100, 
                                   criterion='gini', 
                                   max_depth=None, 
                                   min_samples_split=2, 
                                   min_samples_leaf=1, 
                                   min_weight_fraction_leaf=0.0, 
                                   max_features='auto', 
                                   max_leaf_nodes=None, 
                                   bootstrap=True, 
                                   oob_score=False, 
                                   n_jobs=1, 
                                   random_state=None, 
                                   verbose=0, 
                                   warm_start=False, 
                                   class_weight=None).fit(X_traincvWV, y_traincvWV)

In [92]:
eval_RF_WV_tts = clf_RF_WV.score(X_testcvWV, y_testcvWV)
print(eval_RF_WV_tts)

0.6026


Creating a Submission  
--

In [98]:
testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

Review 0 of 25000
Review 2000 of 25000
Review 4000 of 25000
Review 6000 of 25000
Review 8000 of 25000
Review 10000 of 25000
Review 12000 of 25000
Review 14000 of 25000
Review 16000 of 25000
Review 18000 of 25000
Review 20000 of 25000
Review 22000 of 25000
Review 24000 of 25000


In [99]:
print(np.isnan(testDataVecs).any()) #testando se não há valores que inviabilizam o treinamento
print(np.isfinite(testDataVecs).all())

True
False


In [100]:
from sklearn.preprocessing import Imputer
testDataVecs = Imputer().fit_transform(testDataVecs)

print(np.isnan(testDataVecs).any()) #testando se não há valores que inviabilizam o treinamento
print(np.isfinite(testDataVecs).all())

False
True


In [102]:
# Use the random forest to make sentiment label predictions
result = clf_RF_WV.predict(testDataVecs)
result_prob = clf_RF_WV.predict_proba(testDataVecs)
output = pd.DataFrame(data={"id":test["id"], "sentiment":result,})# "probs":result_prob[:,1]})
# Use pandas to write the comma-separated output file
output.to_csv(os.path.join(datapath,'Word2Vec_AverageVectors.csv'), index=False, quoting=3)

In [103]:
output.head()

,id,sentiment
0,"""12311_10""",1
1,"""8348_2""",1
2,"""5828_4""",0
3,"""7186_2""",0
4,"""12128_7""",0


We found that this produced results much better than chance, but underperformed Bag of Words by a few percentage points.

Since the element-wise average of the vectors didn't produce spectacular results, perhaps we could do it in a more intelligent way? A standard way of weighting word vectors is to apply "tf-idf" weights, which measure how important a given word is within a given set of documents. One way to extract tf-idf weights in Python is by using scikit-learn's TfidfVectorizer, which has an interface similar to the CountVectorizer that we used in Part 1. However, when we tried weighting our word vectors in this way, we found no substantial improvement in performance.

From Words to Paragraphs, Attempt 2: Clustering 
--

Word2Vec creates clusters of semantically related words, so another possible approach is to exploit the similarity of words within a cluster. Grouping vectors in this way is known as "vector quantization." To accomplish this, we first need to find the centers of the word clusters, which we can do by using a clustering algorithm such as K-Means.

In K-Means, the one parameter we need to set is "K," or the number of clusters. How should we decide how many clusters to create? Trial and error suggested that small clusters, with an average of only 5 words or so per cluster, gave better results than large clusters with many words. Clustering code is given below. We use scikit-learn to perform our K-Means.

K-Means clustering with large K can be very slow; the following code took more than 40 minutes on my computer. Below, we set a timer around the K-Means function to see how long it takes.

In [106]:
start = time.time()

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an average of 5 words per cluster
word_vectors = model.syn0
num_clusters = word_vectors.shape[0] / 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans(n_clusters = num_clusters)
idx = kmeans_clustering.fit_predict(word_vectors)

end = time.time()
elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds."

Time taken for K Means clustering:  4767.01249218 seconds.


The cluster assignment for each word is now stored in idx, and the vocabulary from our original Word2Vec model is still stored in model.index2word. For convenience, we zip these into one dictionary as follows:  

In [107]:
# Create a Word / Index dictionary, mapping each vocabulary word to a cluster number
word_centroid_map = dict(zip(model.index2word, idx))

This is a little abstract, so let's take a closer look at what our clusters contain. Your clusters may differ, as Word2Vec relies on a random number seed. Here is a loop that prints out the words for clusters 0 through 9:

Run k-means on the word vectors and print a few clusters  

In [108]:
# Print the first ten clusters
for cluster in xrange(0,10):
    # Print the cluster number
    print "\nCluster %d" % cluster
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( word_centroid_map.values()[i] == cluster ):
            words.append(word_centroid_map.keys()[i])
    print words


Cluster 0
[u'halloweentown']

Cluster 1
[u'blonds', u'dippy', u'buxom', u'uschi', u'daniela', u'jester', u'brandy', u'pam', u'colon', u'playmate', u'tags', u'fuzz', u'annabel', u'brunettes', u'eternally', u'sicko', u'nellie', u'sherri', u'madolyn', u'sheri']

Cluster 2
[u'analyst', u'invents', u'solves', u'kabuki', u'concealing', u'humiliates', u'scout', u'pacifist', u'entrusted', u'morand', u'penguin', u'tasked', u'extortion', u'bogeyman', u'impersonates']

Cluster 3
[u'labelled', u'labeled', u'rightly', u'imaging', u'blueprint']

Cluster 4
[u'calmly', u'winnings', u'hover', u'pocketing', u'ballistic', u'shackles', u'fingerprints', u'awry', u'conveniently', u'miraculously', u'bandages', u'backpack', u'lodged']

Cluster 5
[u'belies', u'reflexive']

Cluster 6
[u'unreal', u'absurd', u'unbelievable', u'ridiculous', u'unrealistic', u'unnecessary', u'ludicrous', u'unconvincing']

Cluster 7
[u'absorbs', u'interventions', u'telepathy', u'inertia', u'maelstrom', u'enraptured']

Cluster 8
[u'd

We can see that the clusters are of varying quality. Some make sense - Cluster 3 mostly contains names, and Clusters 6-8 contain related adjectives (Cluster 6 is my favorite). On the other hand, Cluster 5 is a little mystifying: What do a lobster and a deer have in common (besides being two animals)? Cluster 0 is even worse: Penthouses and suites seem to belong together, but they don't seem to belong with apples and passports. Cluster 2 contains ... maybe war-related words? Perhaps our algorithm works best on adjectives.

At any rate, now we have a cluster (or "centroid") assignment for each word, and we can define a function to convert reviews into bags-of-centroids. This works just like Bag of Words but uses semantically related clusters instead of individual words:

In [109]:
def create_bag_of_centroids(wordlist, word_centroid_map):
    # The number of clusters is equal to the highest cluster index in the word / centroid map
    num_centroids = max(word_centroid_map.values()) + 1
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros(num_centroids, dtype="float32")
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    return bag_of_centroids

The function above will give us a numpy array for each review, each with a number of features equal to the number of clusters. Finally, we create bags of centroids for our training and test set, then train a random forest and extract results:

In [110]:
# ****** Create bags of centroids

# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros((train["review"].size, num_clusters), dtype="float32")

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids(review, word_centroid_map)
    counter += 1

In [113]:
X_traincvCT, X_testcvCT, y_traincvCT, y_testcvCT = train_test_split(train_centroids,
                                                                    train["sentiment"],
                                                                    test_size=0.2,
                                                                    random_state=0)

In [114]:
# Initialize a Random Forest classifier with 100 trees
clf_RF_CT = RandomForestClassifier(n_estimators=100, 
                                   criterion='gini', 
                                   max_depth=None, 
                                   min_samples_split=2, 
                                   min_samples_leaf=1, 
                                   min_weight_fraction_leaf=0.0, 
                                   max_features='auto', 
                                   max_leaf_nodes=None, 
                                   bootstrap=True, 
                                   oob_score=False, 
                                   n_jobs=1, 
                                   random_state=None, 
                                   verbose=0, 
                                   warm_start=False, 
                                   class_weight=None).fit(X_traincvCT, y_traincvCT)

In [115]:
eval_RF_CT_tts = clf_RF_CT.score(X_testcvCT, y_testcvCT)
print(eval_RF_CT_tts)

0.5702


In [116]:
# Repeat for test reviews
test_centroids = np.zeros((test["review"].size, num_clusters), dtype="float32")

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids(review, word_centroid_map)
    counter += 1

In [117]:
result = clf_RF_CT.predict(test_centroids)

# Write the test results
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv(os.path.join(datapath,"BagOfCentroids.csv"), index=False, quoting=3)
print "Wrote BagOfCentroids.csv"

Wrote BagOfCentroids.csv


We found that the code above gives about the same (or slightly worse) results compared to the Bag of Words in Part 1.

Part 4: Comparing deep and non-deep learning methods
--

You may ask: Why is Bag of Words better?

The biggest reason is, in our tutorial, averaging the vectors and using the centroids lose the order of words, making it very similar to the concept of Bag of Words. The fact that the performance is similar (within range of standard error) makes all three methods practically equivalent.  

A few things to try:

First, training Word2Vec on a lot more text should greatly improve performance. Google's results are based on word vectors that were learned out of more than a billion-word corpus; our labeled and unlabeled training sets together are only a measly 18 million words or so. Conveniently, Word2Vec provides functions to load any pre-trained model that is output by Google's original C tool, so it's also possible to train a model in C and then import it into Python.

Second, in published literature, distributed word vector techniques have been shown to outperform Bag of Words models. In this paper, an algorithm called Paragraph Vector is used on the IMDB dataset to produce some of the most state-of-the-art results to date. In part, it does better than the approaches we try here because vector averaging and clustering lose the word order, whereas Paragraph Vectors preserves word order information.

What is Deep Learning?

The term "deep learning" was coined in 2006, and refers to machine learning algorithms that have multiple non-linear layers and can learn feature hierarchies [1].

Most modern machine learning relies on feature engineering or some level of domain knowledge to obtain good results. In deep learning systems, this is not the case -- instead, algorithms can automatically learn feature hierarchies, which represent objects in increasing levels of abstraction. Although the basic ingredients of many deep learning algorithms have been around for many years, they are currently increasing in popularity for many reasons, including advances in compute power, the falling cost of computing hardware, and advances in machine learning research.

Deep learning algorithms can be categorized by their architecture (feed-forward, feed-back, or bi-directional) and training protocols (purely supervised, hybrid, or unsupervised) [2]. 

Some good background materials include:

[1] "Deep Learning for Signal and Information Processing", by Li Deng and Dong Yu (out of Microsoft)

[2] "Deep Learning Tutorial" (2013 Presentation by Yann LeCun and Marc'Aurelio Ranzato)

Where Does Word2Vec Fit In?

Word2Vec works in a way that is similar to deep approaches such as recurrent neural nets or deep neural nets, but it implements certain algorithms, such as hierarchical softmax, that make it computationally more efficient.  

See Part 2 of this tutorial for more on Word2Vec, as well as this paper: Efficient Estimation of Word Representations in Vector Space

In this tutorial, we use a hybrid approach to training -- consisting of an unsupervised piece (Word2Vec) followed by supervised learning (the Random Forest). 

Libraries and Packages 

The lists below should in no way be considered exhaustive.

In Python:

Theano offers very low-level, nuts and bolts functionality for building deep learning systems. You can also find some good tutorials on their site.  
Caffe is a deep learning framework out of the Berkeley Vision and Learning Center.  
Pylearn2 wraps Theano and seems slightly more user friendly.  
OverFeat was used to win the Kaggle Cats and Dogs competition.  


More Tutorials  
The O'Reilly Blog has a series of deep learning articles and tutorials:  

http://radar.oreilly.com/2014/07/what-is-deep-learning-and-why-should-you-care.html  
http://radar.oreilly.com/2014/07/how-to-build-and-run-your-first-deep-learning-network.html  
Webcast: How to Get Started with Deep Learning in Computer Vision  
There are several tutorials using Theano as well.  

If you want to dive into the weeds of creating a neural network from scratch, check out Geoffrey Hinton's Coursera course.

For NLP, check out this recent lecture at Stanford: http://techtalks.tv/talks/deep-learning-for-nlp-without-magic-part-1/58414/  

This free, online book also introduces neural nets for deep learning: http://neuralnetworksanddeeplearning.com/  